# Table of Contents
1  Initialize and create an interactive map
2  Add basemaps
3  Add WMS and XYZ tile layers
4  Search and add Earth Engine data to layer
5  Search Earth Engine API documentation
6  Use Inspector tool and Drawing tools
7  Convert JavaScripts to Python
8  Use and upload local shapefiles
9  Download a image from google earth engine

In [1]:
# -------------------------------------------------------
# 1 import ee and Initialize the map
# -------------------------------------------------------
import ee, geemap

# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map


Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [2]:
# -------------------------------------------------------
# 2 Add basemaps and split Map
# -------------------------------------------------------
# geemap.Map()
Map.add_basemap('HYBRID')

In [3]:
# -------------------------------------------------------
# 2 Add basemaps and split Map
# -------------------------------------------------------
Map.split_map(left_layer='SATELLITE', right_layer='Esri.WorldImagery')
# Map.setCenter(29.68, 116.015,zoom=8)

In [9]:
# -------------------------------------------------------
# 3  Add WMS and XYZ tile layers
# -------------------------------------------------------

url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Satellite', attribution='Google')

In [2]:
# -------------------------------------------------------
# 4  Search and add Earth Engine data to layer
# -------------------------------------------------------

# https://developers.google.com/earth-engine/datasets/catalog

image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}
Map.setCenter(-122.1899, 37.5010, 8)      # 按地图视图按经纬度坐标来
Map.addLayer(image, vizParams, 'false color composite')
# bands参数需要一个有且只含三个元素的列表，列表里每一个参数为影像的一个波段的名称
# 用这三个波段来映射到RGB的红绿蓝三种颜色中
# min,max设置显示波段值的颜色范围
# gamma，即用于gamma矫正的一些参数，以矫正图像

In [2]:
# -------------------------------------------------------
# 5  Search Earth Engine API documentation
# -------------------------------------------------------
#     https://developers.google.com/earth-engine/guides
#     https://code.earthengine.google.com/?accept_repo=users/nclinton/EE101
# -------------------------------------------------------
# 6  Use Inspector tool and Drawing tools
# -------------------------------------------------------
# Map.draw_features
# Map.draw_last_feature
feature = Map.draw_last_feature
roi = feature.geometry()
print(roi)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Feature.geometry",
    "arguments": {
      "feature": {
        "functionInvocationValue": {
          "functionName": "Feature",
          "arguments": {
            "geometry": {
              "functionInvocationValue": {
                "functionName": "GeometryConstructors.Polygon",
                "arguments": {
                  "coordinates": {
                    "constantValue": [
                      [
                        [
                          115.784283,
                          28.356118
                        ],
                        [
                          115.784283,
                          29.41424
                        ],
                        [
                          116.825396,
                          29.41424
                        ],
                        [
                          116.825396,
                          28.356118
                        ],
          

In [4]:
# 6  Use Inspector tool and Drawing tools -> example
# image = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(roi).sort('CLOUD_COVER').first()
# 将数据按照云量数据大小排序并且选取第一个，按照roi裁剪
image = ee.ImageCollection("LANDSAT/LO08/C01/T1").filterBounds(roi).sort('CLOUD_COVER').first()
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
   'min': 0.0,
   'max': 30000.0
}
Map.addLayer(image,vizParams, 'landsat8')
# Map.addLayer(image,{'bands':['SR_B5','SR_B4','SR_B3']}, 'landsat9')

In [9]:
# props = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').propertyNames().getInfo()
# print(props)

date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss').getInfo()
print(f'date :{date}')

date :2015-10-18 02:46:58


In [13]:
# -------------------------------------
# 7  Convert JavaScripts to Python
# -------------------------------------

js_snippet = """
// This example demonstrates the use of the Landsat 8 Collection 2, Level 2
// QA_PIXEL band (CFMask) to mask unwanted pixels.

function maskL8sr(image) {
  // Bit 0 - Fill
  // Bit 1 - Dilated Cloud
  // Bit 2 - Cirrus
  // Bit 3 - Cloud
  // Bit 4 - Cloud Shadow
  var qaMask = image.select('QA_PIXEL').bitwiseAnd(parseInt('11111', 2)).eq(0);
  var saturationMask = image.select('QA_RADSAT').eq(0);

  // Apply the scaling factors to the appropriate bands.
  var opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  var thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);

  // Replace the original bands with the scaled ones and apply the masks.
  return image.addBands(opticalBands, null, true)
      .addBands(thermalBands, null, true)
      .updateMask(qaMask)
      .updateMask(saturationMask);
}

// Map the function over one year of data.
var collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                     .filterDate('2020-01-01', '2021-01-01')
                     .map(maskL8sr);

var composite = collection.median();

// Display the results.
Map.setCenter(-4.52, 40.29, 7);  // Iberian Peninsula
Map.addLayer(composite, {bands: ['SR_B4', 'SR_B3', 'SR_B2'], min: 0, max: 0.3});

"""
geemap.js_snippet_to_py(js_snippet, add_new_cell=False, import_ee=False, import_geemap=False, show_map=False)

None


NameError: name 'parseInt' is not defined

In [ ]:
# --------------------------------------------
# 8  Use and upload local shapefiles
# --------------------------------------------

# local shape file 
f_shp = r'D:\08.专业数据\A1_database\A2_images\00.基础地理数据\stations_2k_dd\stations_2k_gcs.shp' #添加面shp（文件名不能有“_”和中文）

# convert local shape file to ee.feature
shp_ee = geemap.shp_to_ee(f_shp)

# add ee.feature to the map
# Map.addLayer(eeObject, visParams, name, shown, opacity)
Map.addLayer(shp_ee, {}, 'Climate stations')

In [2]:
# --------------------------------------------
# 9  Download a image from google earth engine
# Total request size must be less than or equal to 50331648 bytes
# --------------------------------------------
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
fidx = image.get('system:index').getInfo()

f_img = 'D:\03.Temporary' + os.sep + fidx + '.tif'
geemap.ee_export_image(image, filename=f_img, scale=500,file_per_band=True)
# geemap.ee_export_image_to_drive(image, description='landsat8', folder='export', scale=30)

Generating URL ...
An error occurred while downloading.
Total request size (893902500 bytes) must be less than or equal to 50331648 bytes.


In [ ]:
# --------------------------------------------
# 9  Download a image from google earth engine to google drive
# 
# --------------------------------------------
geemap.ee_export_image_to_drive(image, description=fidx, folder='ee_export', scale=30)